In [2]:
#--------------------------------------------------------------------------------------------------------
# Importing Python Modules
import csv
import pandas as pd
import pathlib
import numpy as np
from math import sqrt, pi, log, e
from enum import Enum
import scipy.stats as stat
from scipy.stats import norm
from scipy import stats
import sympy
from sympy.stats import Normal, cdf
from sympy import init_printing

# The Black Model (Black-76)
Variant of the Black-Scholes option pricing model, the black model's use is for prcing options on derivatives like futures, interest rate cap and floors, and swaptions. First presented by Fisher Black in a paper written in 1976. Black-76 can be classified as a log-normal forward model.

#### Formula:
- Call $= e^{(-rT)}[FN(d_1)-KN(d_2)]$
- Put $= e^{(-rT)}[KN(-d_2)-FN(-d_1)]$
- Where:
    - $d_1 = \frac{ln(F/K)+(\sigma^2/2)T}{\sigma \sqrt{T}}$
    - $d_2 = d_1 - \sigma \sqrt{T}$
    - $ln =$ Log-normal Distribution
    - $N =$ Cumulative Normal Distributuion
    - $F =$ Underlying Forward Price
    - $K =$ Strike Price
    - $T =$ Time Until Expiration
    - $\sigma =$ Volatility
    - $r =$ Risk Free Interest Rate
   
## Example:
- Pricing /ES Futures (S&P500 Index) Options:
    - Where:
        - (Foward Price) $F$ = 3925
        - (Strike Price) $K$ = 3950
        - (Time to Exp) $T$ = 6 Days
        - (Volatility) <font size="4.5">$\sigma$</font> = 14%
        - (Risk Free Rate) $r$ = 0.19% 
        
#### First let's solve for Theortical Price of a Call Option:

In [3]:
#------------------------------------
# Model
def b76(f, k, r, sigma, t, option_type):
    d1 = (log(f / k) + (sigma ** 2 / 2) * t) / (sigma * sqrt(t))             # Derivative 1
    d2 = d1 - sigma * sqrt(t)                                                # Derivative 2
    if option_type == 'call':         
        price = e**(-r * t) * (f * norm.cdf(d1) - k * norm.cdf(d2))          # Theoratical Price
        delta = (e**(r*t))*(norm.cdf(d1))                                    # Delta 
        p_density = 1 / sqrt(2 * np.pi) * e**(-d1 ** 2 * 0.5)                # Probabiltiy Density
        gamma = p_density / (f * sigma * sqrt(t))                            # Gamma
        vega = (f*(sqrt(t))*(p_density))                                     # Vega
        theta = -(f*sigma*norm.pdf(d1))/2*sqrt(t)-r*k*e**(-r*t)*norm.cdf(d2) # Theta
    elif option_type == 'put':
        price = e**(-r * t) * (k * norm.cdf(-d2) - f * norm.cdf(-d1))   
        delta = (e**(r*t))*(norm.cdf(d1)-1)
        p_density = 1 / sqrt(2 * np.pi) * e**(-d1 ** 2 * 0.5)
        gamma = p_density / (f * sigma * sqrt(t))
        vega = (f*(sqrt(t))*(p_density))
        theta = -(f*sigma*norm.pdf(d1))/2*sqrt(t)-r*k*e**(-r*t)*norm.cdf(d2)
    return price,delta,gamma,vega,theta
#------------------------------------

In [8]:
#------------------------------------
# Inputs
F = 3925
K = 3950
T = 6.58 / 365 # Divided by 365 for yearly format
sigma = 0.17 # 17% Volatility so 0.17 sigma
r = 0.02 # Risk free interest rate is 2%
#------------------------------------
output = b76(F,K,r,sigma,T,'call')
print("Therotical Call Option Price: ${}".format(output[0]))

Therotical Call Option Price: $24.723194261590088


#### Option Greeks:

In [7]:
#------------------------------------
# Decimal manipulation function for cleaning the output's
def mDP(num,d_p):
    for _ in range(abs(d_p)):
        if d_p>0:
            num *= 10;
        else:
            num /= 10.;
    return float(num)
#------------------------------------
delta = int(mDP(output[1],2)) # The delta is 0.39 aka 39 delta so turn to integer
gamma = round(mDP(output[2],2),3) # round gamma and adjust decimals for .xxx
vega = round(float(mDP(output[3],-2)),3) # round, float, and adjust decimals for x.xxx
theta = round(mDP(output[4],-2),3) # round and adjust decimals for xx.xxx
greeks = ["Delta: {}".format(delta),"Gamma: {}".format(gamma),"Vega: {}".format(vega),"Theta: {}".format(theta)]
for greek in greeks:
    print(greek)

Delta: 39
Gamma: 0.43
Vega: 2.029
Theta: -0.477
